<h2>Setup & Import</h2>

In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# Import data and setup dataframe
df = pd.read_csv('data/T_T100_SEGMENT_ALL_CARRIER.csv')

In [12]:
# Import lookup tables
carrier_classes = pd.read_csv('data/L_CARRIER_GROUP_NEW.csv')
area_codes_table = pd.read_csv('data/L_WORLD_AREA_CODES.csv')
aircraft_types = pd.read_csv('data/L_AIRCRAFT_TYPE.csv')

city_markets = pd.read_csv('data/L_CITY_MARKET_ID.csv')
service_classes = pd.read_csv('data/L_SERVICE_CLASS.csv')
aircraft_groups = pd.read_csv('data/L_AIRCRAFT_GROUP.csv')
aircraft_congifs = pd.read_csv('data/L_AIRCRAFT_CONFIG.csv')

<h2>Data Cleansing</h2>

<h4>Carrier Classes</h4>

In [9]:
# Create simplified list of carrier class descriptions
carrier_classes_modified = []
for carrier_type in carrier_classes['Description']:
    index = carrier_type.index('Carrier')
    substr = carrier_type[:index-1]
    carrier_classes_modified.append(substr)

carrier_classes['Carrier Class'] = carrier_classes_modified

In [11]:
#Inspect updated carrier_classes
carrier_classes

,Code,Description,Carrier Class
0,0,Foreign Carriers,Foreign
1,1,Large Regional Carriers (carriers with annual ...,Large Regional
2,2,National Carriers (carriers with annual revenu...,National
3,3,Major Carriers (carriers with annual revenue o...,Major
4,4,Medium Regional Carriers (carriers with annual...,Medium Regional
5,5,Small Certificated Carriers (carrier holding c...,Small Certificated
6,6,Commuter Carriers (air taxi operator which per...,Commuter
7,7,All Cargo Carriers operating under cerificates...,All Cargo
8,9,Commuter Carriers (Air Taxi providing Essentia...,Commuter


In [7]:
# Replace carrier groups with descriptions from modified lookup table
df = df.merge(carrier_classes, left_on='CARRIER_GROUP_NEW', right_on='Code', how='left')
df = df.drop(columns=['Code', 'Description', 'CARRIER_GROUP_NEW'])

<h4>World Area Codes (WAC)</h4>

In [ ]:
# Replace WAC with descriptions from lookup table
df = df.merge(area_codes_table, left_on='ORIGIN_WAC', right_on='Code', how='left')
df['ORIGIN_WAC'] = df['Description']
df = df.drop(columns=['Code', 'Description'])

df = df.merge(area_codes_table, left_on='DEST_WAC', right_on='Code', how='left')
df['DEST_WAC'] = df['Description']
df = df.drop(columns=['Code', 'Description'])

<h4>Aircraft type</h4>

In [16]:
aircraft_types

,Code,Description
0,7,Aero Commander 200
1,8,Aero Macchi AL-60
2,9,Aeronca 7-AC
3,10,Beech Bonanza 35A/C/D/E/G/H/J/K/S/V/ 36A
4,20,Bellanca CH-300
...,...,...
439,887,B787-800 Dreamliner
440,888,Boeing 737-900ER
441,889,B787-900 Dreamliner
442,890,Antonov 225 (6 Engine)


<h2>Add Load Factor Metric</h2>

In [ ]:
#Add Load Factor column on flight level
df['Load Factor'] = df['PASSENGERS'] / df['SEATS']
df['Load Factor'] = df['Load Factor'].fillna(0)

<h2>Display Final Table</h2>

In [ ]:
df[:10]